



# Andreev Retroreflection
## Viktor Könye

In [1]:
# imports
from numpy import *

from scipy.sparse import coo_matrix
import scipy as sc
from scipy.special import jv

from ipywidgets import *
from JSAnimation import IPython_display
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib inline

from traitlets.config.manager import BaseJSONConfigManager
path = "/home/viktor/.jupyter/nbconfig"
cm = BaseJSONConfigManager(config_dir=path)
cm.update('livereveal', {
              'theme': 'sky',
              'transition': 'zoom'
                });


### Kernel Polynomial Method (KPM)

\begin{equation}
\mathrm{e}^{-itx} = J_0(t)+2\sum\limits_{m=1}^\infty (-i)^mJ_m(t)T_m(x)
\end{equation}

* $x\in[-1,1]$
* $J_m$: Bessel functions of the first kind
* $T_m$: Chebyshev polynomials of the first kind

* Recurrence relation for Chebyshev polynomials:

\begin{align}
T_{m+1}(x)+T_{m-1}(x)&=2xT_m(x) \\
T_0(x)&=1\\
T_1(x)&=x
\end{align}

* Time evolution of $\mid\psi(0)\rangle$:

\begin{equation}
\mid\psi(t)\rangle=\mathrm{e}^{-itH} \mid\psi(0)\rangle
\end{equation}

* Normalized Hamiltonian $\rightarrow$ previous expansion can be used

\begin{equation}
\mid\psi(t)\rangle = J_0(t)\mid\psi(0)\rangle+2\sum\limits_{m=1}^\infty (-i)^mJ_m(t)T_m(H)\mid\psi(0)\rangle
\end{equation}

\begin{align}
T_0(H)\mid\psi(0)\rangle&=\mid\psi(0)\rangle\\
T_1(H)\mid\psi(0)\rangle&=H\mid\psi(0)\rangle \\
T_{m+1}(H)\mid\psi(0)\rangle&=2HT_m(H)\mid\psi(0)\rangle-T_{m-1}(H)\mid\psi(0)\rangle
\end{align}

In [2]:
def infinitesimal_timestep(H,vo,dt):
    prevPrev=vo
    prev=H*vo
    v = jv(0,dt) * prevPrev + 2 * (-1j) * jv(1,dt) * prev
    m = 2
    while (jv(m,dt) > 1e-16 or jv(m+1,dt) > 1e-16):
        now = 2 * H * prev - prevPrev
        v = v + 2 * (-1j)**m * jv(m,dt) * now
        prevPrev = prev
        prev = now
        m += 1
    return v    

In [3]:
# Time evolution
def time_evolution(H,vo,dt,tMax,disp):
    t = arange(0,tMax+dt,dt)
    numberOfIter = int(tMax/dt)
    x = linspace(-1,1,int(vo.shape[0]/2))
    vt = zeros((vo.shape[0],numberOfIter+1))*1j
    vt[:,0] = vo.toarray()[:,0]
    v = vo
    
    progress = FloatProgress(min=0, max=numberOfIter); # progress bar
    if disp == True:
        display(progress) # progress bar
    for i in range(numberOfIter):
        v = infinitesimal_timestep(H,v,dt)
        sc.sparse.linalg.norm(v)
        v = v/sc.sparse.linalg.norm(v) # norm regularization
        vt[:,i+1] = v.toarray()[:,0]
        progress.value = i
    progress.value = numberOfIter
        
    return int(numberOfIter+1), x, vt

### Bogoljubov-de Gennes Model

* MFT


* BdG Hamiltonian
\begin{equation}
\pmatrix{h &\Delta \\ -\Delta^* & -h^*}
\end{equation}
* Particle hole symmetry

### 1D Chain

In [4]:
def hamiltonian_1d(M,epsilon,delta,a,b,k,sigma,xo): # M legyen páros
    gamma = 0.5
    eps = array(range(int(M/2)-a,int(M/2)+b))  
    He = (coo_matrix((epsilon/2 * ones(a+b),(eps,eps)), shape=(2*M,2*M)) - 
        coo_matrix((gamma * ones(M-1),(array(range(M-1)),array(range(1,M)))), shape=(2*M,2*M)) - 
         coo_matrix(([gamma],([0],[M-1])), shape=(2*M,2*M)))
    Hh = (coo_matrix((-epsilon/2 * ones(a+b),(M+eps,M+eps)), shape=(2*M,2*M)) + 
        coo_matrix((gamma * ones(M-1),(M+array(range(M-1)),M+array(range(1,M)))), shape=(2*M,2*M)) + 
         coo_matrix(([gamma],([M],[2*M-1])), shape=(2*M,2*M)))
    D =  coo_matrix((delta * ones(a+b),(M+eps,eps)), shape=(2*M,2*M))
 
    H = He + Hh + D
    H = H + H.H
    
    vo = coo_matrix((exp(1j * k * arange(M) - (arange(M)-xo)**2/(2*sigma**2)),(arange(M),arange(M)*0)),shape=(2*M,1))
    vo = vo/sc.sparse.linalg.norm(vo)

    potential = zeros(M)
    potential[eps]=epsilon
    sup = zeros(M)
    sup[eps]=delta            

    #print(H.toarray())
    #print(vo.toarray())
    return H, vo, potential, sup

#hamiltonian_1d(4,0,1,1,1,0,10,0);

In [5]:
def plotter():
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1.5*max(abs(vo.toarray())**2)[0], 1.5*max(abs(vo.toarray())**2)[0]])
    ax.axis('off')
    ax.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
    ax.tick_params(axis='y', which='both', right='off', left='off', labelleft='off')
    line, = ax.plot(x,abs(vt[0:int(vt.shape[0]/2),0])**2, lw=1, c='blue')
    linh, = ax.plot(x,-abs(vt[int(vt.shape[0]/2):vt.shape[0],0])**2, lw=1, c='blue')
    pot, = ax.plot(x,potential*0.5*max(abs(vo.toarray())**2), lw=1, c='green')
    supe, = ax.plot(x,sup*2*max(abs(vo.toarray())**2), lw=1, c='red')   
    return fig,ax,line,linh,

def init():
    line.set_data([], [])
    return line,

def animate(i):
    line.set_data(x,abs(vt[0:int(vt.shape[0]/2),i])**2)
    linh.set_data(x,-abs(vt[int(vt.shape[0]/2):vt.shape[0],i])**2)
    return line, linh,

#### Free electron

In [ ]:
H , vo , potential, sup = hamiltonian_1d(M=1000,epsilon=0,delta=0,a=0,b=250,k=1.6,sigma=50,xo=300)
steps , x , vt = time_evolution(H,vo,dt=2,tMax=1000,disp=False)

fig, ax, line, linh, = plotter()
animation.FuncAnimation(fig, animate, init_func=init, frames=int(steps), interval=25, blit=True)

#### Free electron

In [ ]:
H , vo , potential, sup = hamiltonian_1d(M=1000,epsilon=0,delta=0,a=0,b=250,k=1.6,sigma=10,xo=300)
steps , x , vt = time_evolution(H,vo,dt=2,tMax=1000,disp=False)

fig, ax, line, linh, = plotter()
animation.FuncAnimation(fig, animate, init_func=init, frames=int(steps), interval=25, blit=True)

#### Potential

In [ ]:
H , vo , potential, sup = hamiltonian_1d(M=1000,epsilon=0.6,delta=0,a=100,b=100,k=1.6,sigma=30,xo=300)
steps , x , vt = time_evolution(H,vo,dt=2,tMax=1000,disp=False)

fig, ax, line, linh, = plotter()
animation.FuncAnimation(fig, animate, init_func=init, frames=int(steps), interval=25, blit=True)

#### Potential

In [ ]:
H , vo , potential, sup = hamiltonian_1d(M=1000,epsilon=1.1,delta=0,a=100,b=100,k=1.6,sigma=30,xo=300)
steps , x , vt = time_evolution(H,vo,dt=2,tMax=400,disp=False)

fig, ax, line, linh, = plotter()
animation.FuncAnimation(fig, animate, init_func=init, frames=int(steps), interval=25, blit=True)

#### Potential

In [ ]:
H , vo , potential, sup = hamiltonian_1d(M=1000,epsilon=1.1,delta=0,a=1,b=1,k=1.6,sigma=30,xo=300)
steps , x , vt = time_evolution(H,vo,dt=2,tMax=400,disp=False)

fig, ax, line, linh, = plotter()
animation.FuncAnimation(fig, animate, init_func=init, frames=int(steps), interval=25, blit=True)

#### Superconductor

In [ ]:
H , vo , potential, sup = hamiltonian_1d(M=1000,epsilon=0,delta=0.2,a=100,b=100,k=1.6,sigma=30,xo=300)
steps , x , vt = time_evolution(H,vo,dt=2,tMax=400,disp=False)

fig, ax, line, linh, = plotter()
animation.FuncAnimation(fig, animate, init_func=init, frames=int(steps), interval=25, blit=True)

#### Superconductor

In [ ]:
H , vo , potential, sup = hamiltonian_1d(M=1000,epsilon=0,delta=0.4,a=50,b=50,k=2,sigma=30,xo=300)
steps , x , vt = time_evolution(H,vo,dt=2,tMax=800,disp=False)

fig, ax, line, linh, = plotter()
animation.FuncAnimation(fig, animate, init_func=init, frames=int(steps), interval=25, blit=True)

### 2D Square lattice

In [6]:
def hamiltonian_2d(M,epsilon,delta,a,b,k,sigma,xo): # M legyen páros
    gamma = 0.2
    eps = array(range(a*M,b*M)) 
    eeps=0.2
    He = (coo_matrix((eeps * ones(M*M),(array(range(M*M)),array(range(M*M)))), shape=(2*M*M,2*M*M)) + 
          coo_matrix((epsilon/2 * ones((b-a)*M),(eps,eps)), shape=(2*M*M,2*M*M)) -
        coo_matrix((gamma * ones(M*M-1),(array(range(1,M*M)),array(range(M*M-1)))), shape=(2*M*M,2*M*M)) +
        coo_matrix((gamma * ones(M-1),(arange(M,M*M,M),arange(M-1,M*M-1,M))), shape=(2*M*M,2*M*M)) -
        coo_matrix((gamma * ones(M*M-M),(array(range(M,M*M)),array(range(M*M-M)))), shape=(2*M*M,2*M*M)))
    Hh = (coo_matrix((-eeps * ones(M*M),(M*M+array(range(M*M)),M*M+array(range(M*M)))), shape=(2*M*M,2*M*M)) +
        coo_matrix((-epsilon/2 * ones((b-a)*M),(M*M+eps,M*M+eps)), shape=(2*M*M,2*M*M)) + 
        coo_matrix((gamma * ones(M*M-1),(M*M+array(range(1,M*M)),M*M+array(range(M*M-1)))), shape=(2*M*M,2*M*M)) -
        coo_matrix((gamma * ones(M-1),(arange(M*M+M,2*M*M,M),arange(M*M+M-1,2*M*M-1,M))), shape=(2*M*M,2*M*M)) +
        coo_matrix((gamma * ones(M*M-M),(M*M+array(range(M,M*M)),M*M+array(range(M*M-M)))), shape=(2*M*M,2*M*M))) 
    D =  coo_matrix((delta * ones((b-a)*M),(M*M+eps,eps)), shape=(2*M*M,2*M*M)) 
 
    H = He + Hh + D
    H = H + H.H
    
    y = arange(M*M) % M
    x = floor(arange(M*M)/M)
    vo = coo_matrix((exp(1j * (k[0] * x + k[1] * y) - ((x-xo[0])**2+(y-xo[1])**2)/(2*sigma**2)),(arange(M*M),arange(M*M)*0)),shape=(2*M*M,1))
    vo = vo/sc.sparse.linalg.norm(vo)

    potential = zeros(M*M)
    potential[eps]=epsilon
    sup = zeros(M*M)
    sup[eps]=delta            

    #print(H.toarray())
    #print(vo.toarray())
    return H, vo, potential, sup

hamiltonian_2d(2,0,0,1,2,[1.5,1.5],10,[0,0]);

In [7]:
def init_2d(a,b):
    global fig, ax, ax2, mat, mat2, M
    fig, (ax,ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 6))
    ax.axis('off')
    ax.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
    ax.tick_params(axis='y', which='both', right='off', left='off', labelleft='off')
    ax2.axis('off')
    ax2.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
    ax2.tick_params(axis='y', which='both', right='off', left='off', labelleft='off')
    M = int(sqrt(vt.shape[0]/2))
    mat = ax.imshow(reshape(abs(vt[0:int(vt.shape[0]/2),0])**2,(M,M)).T, cmap='afmhot')
    mat2 = ax2.imshow(reshape(abs(vt[0:int(vt.shape[0]/2),0])**2,(M,M)).T, cmap='afmhot')
    #mat2 = ax2.imshow(reshape(log(abs(vt[int(vt.shape[0]/2):M*M*2,100])**2+1),(M,M)).T, cmap='afmhot')
    #mat2 = ax2.imshow(reshape(log(abs(vt[0:int(vt.shape[0]/2),0])**2+1),(M,M)).T, cmap='afmhot')
    mat2.set_data(reshape(abs(vt[int(vt.shape[0]/2):M*M*2,0])**2,(M,M)).T)

    if (max(abs(potential)) > 1e-6):
        ax.plot(ones((M,1))*a,arange(M), lw=1, c='green'); ax.plot(ones((M,1))*b,arange(M), lw=1, c='green');
        ax2.plot(ones((M,1))*a,arange(M), lw=1, c='green'); ax2.plot(ones((M,1))*b,arange(M), lw=1, c='green');
    if (max(abs(sup)) > 1e-6):
        ax.plot(ones((M,1))*a,arange(M), lw=1, c='red'); ax.plot(ones((M,1))*b,arange(M), lw=1, c='red');
        ax2.plot(ones((M,1))*a,arange(M), lw=1, c='red'); ax2.plot(ones((M,1))*b,arange(M), lw=1, c='red');
    return

def animate_2d(i):
    mat.set_data(reshape(abs(vt[0:int(vt.shape[0]/2),i])**2,(M,M)).T)
    mat2.set_data(reshape(abs(vt[int(vt.shape[0]/2):M*M*2,i])**2,(M,M)).T)
    #mat2.set_data(reshape(log(abs(vt[int(vt.shape[0]/2):M*M*2,i])**2+1),(M,M)).T)
    return

#### Free electron

In [ ]:
a=150; b=300;
H , vo , potential, sup = hamiltonian_2d(300,0,0,a,b,[1.6,1.6],15,[100,100])
steps , x , vt = time_evolution(H,vo,dt=4,tMax=400,disp=False)
    
init_2d(a,b)
animation.FuncAnimation(fig, animate_2d, frames=int(steps), interval=25, blit=False)

#### Potential

In [ ]:
a=150; b=300;
H , vo , potential, sup = hamiltonian_2d(300,0.4,0,a,b,[1.6,1.6],15,[100,100])
steps , x , vt = time_evolution(H,vo,dt=4,tMax=500,disp=False)
    
init_2d(a,b)
animation.FuncAnimation(fig, animate_2d, frames=int(steps), interval=25, blit=False)

#### Superconductor

In [ ]:
a=150; b=300
H , vo , potential, sup = hamiltonian_2d(300,0,0.1,a,b,[1.5,0],20,[100,150])
steps , x , vt = time_evolution(H,vo,dt=4,tMax=300,disp=False)
    
init_2d(a,b)
animation.FuncAnimation(fig, animate_2d, frames=int(steps), interval=25, blit=False)

#### Superconductor

In [ ]:
a=150; b=300;
H , vo , potential, sup = hamiltonian_2d(300,0,0.2,a,b,[1.2,1.2],20,[90,90])
steps , x , vt = time_evolution(H,vo,dt=5,tMax=500,disp=False)
    
init_2d(a,b)
animation.FuncAnimation(fig, animate_2d, frames=int(steps), interval=25, blit=False)

https://github.com/viktorkonye/Andreev-Retroreflection